<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_01_building_a_multi_round_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Get Started by Building a Multi-Round Chat with Agently Workflow! `#Agently_Workflow_Showcase_Series - 01`

## Demo Description

**Author:** Agently Team

**Series Link**:

[Next Document]: [02 - Using Condition to Branch Off](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_02_using_condition_to_branch_off_inbpy.ipynb)

**Description:**

We're glad to introduce a brand new feature of **_<font color = "red">Agent</font><font color = "blue">ly</font>_** v3.2 to you all: **`Agently Workflow`**!

With this new feature, you can arrange and manage your LLMs based application workflow **in just 3 steps, simple and easy:**

1. Define and program your application logic into different `workflow chunks`;
2. Connect chunks using `chunk.connect_to()` in orders; (Loop and condition judugment supported)
3. Start the workflow using `workflow.start()`.

This showcase is the first one of the **`Agently Workflow Showcase Series`**, more showcases will coming soon.

In this case, we will show you how to use **`Agently Workflow`** to create a multi-round chat workflow:

我们非常高兴能够向您介绍**_<font color = "red">Agent</font><font color = "blue">ly</font>_** v3.2版本推出的全新功能：**`Agently Workflow`**！

使用这项全新的功能，只需要一二三步走，您就能够轻松惬意地编排管理您的语言模型应用工作流了：

1. 在`workflow chunks`切块中编写您的应用工作流中的单块工作逻辑（如输入、判断、请求执行、数据存取等）；
2. 使用`chunk.connect_to()`方法将切块按您想要的工作顺序进行连接（支持环状连接、条件分支等复杂连接关系）；
3. 通过`workflow.start()`开始运行工作流。

本案例是**`Agently Workflow系列案例`**的第一个案例，后续我们还将更新更多相关案例。

在本案例中，我们将向您演示如何使用**Agently Workflow**功能创建一个多轮对话的工作流：

## Step 1: Install Packages

In [ ]:
!pip install -q -U Agently

> ℹ️ Agently Workflow is a new feature in version >= `3.2.0`

## Step 2: Create a Multi-Round Chat Workflow in the Basic Way

### Workflow Graph

<img width = "640" src = "https://github.com/Maplemx/Agently/assets/4413155/6da2539d-afd8-4b57-9af8-0135e6afd61d"></img>

### Code

In [ ]:
import Agently
import logging

agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

# Step 0. Create Workflow Instance and Agent Instance
workflow = Agently.Workflow()
# reset logger level to WARNING
workflow.settings.set("logger", logging.getLogger("Workflow").setLevel(logging.WARNING))
agent = agent_factory.create_agent()

# Step 1. Create Chunks
## Start Chunk
start_chunk = workflow.schema.create_chunk(
    title = "Multi-Round Chat",
    type = "Start"
)

## User Input Chunk
user_input_chunk = workflow.schema.create_chunk(
    title = "User Input",
    handles = {
        "outputs": [{ "handle": "user_input" }],
    },
    executor = lambda inputs, storage: {
        "user_input": input("[User]: ")#<- You can change it to an API/WS request
    },# You can use a lambda function as an executor
)

## Assistant Reply Chunk
# You can also define an normal executor function to handle more complex logic
# get data from input handles by `inputs["<handle name>"]`
# get data from cache storage by `storage.get("<key>")`
# save data to cache storage by `storage.save("<key>", <value>)`
def assistant_reply_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    reply = (
        agent
            .chat_history(chat_history)
            .input(inputs["user_input"])
            .start()
    )
    print("[Assistant]: ", reply)
    return { "assistant_reply": reply }
assistant_reply_chunk = workflow.schema.create_chunk(
    title = "Assistant Reply",
    handles = {
        "inputs": [{ "handle": "user_input" }],
        "outputs": [{ "handle": "assistant_reply" }],
    },
    executor = assistant_reply_executor,
)

## Update Chat History Chunk
def update_chat_history_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    chat_history.append({ "role": "user", "content": inputs["user_input"] })
    chat_history.append({ "role": "assistant", "content": inputs["assistant_reply"] })
    storage.set("chat_history", chat_history)
    return
update_chat_history_chunk = workflow.schema.create_chunk(
    title = "Update Chat History",
    handles = {
        "inputs": [
            { "handle": "user_input" },
            { "handle": "assistant_reply" },
        ],
    },
    executor = update_chat_history_executor,
)

# Step 2. Connect Chunks in Orders
start_chunk.connect_to(user_input_chunk)
user_input_chunk.handle("user_input").connect_to(assistant_reply_chunk.handle("user_input"))
user_input_chunk.handle("user_input").connect_to(update_chat_history_chunk.handle("user_input"))
assistant_reply_chunk.handle("assistant_reply").connect_to(update_chat_history_chunk.handle("assistant_reply"))
## Created a loop by connect tail chunk to head chunk
update_chat_history_chunk.connect_to(user_input_chunk)

# Step 3. Start Workflow
workflow.start()

User Input
User Input
[User]: Hi, my name is Max
[Assistant]:  Hi Max, how can I help you today?
[User]: Tell me one tip of Python
[Assistant]:  **Tip:** Use a type hinting system to improve code readability and maintainability.

Python supports type hinting through the `typing` module, allowing you to specify the expected types of function arguments and return values. This helps catch potential type errors at compile time and improves collaboration by making it clear what data types are expected. Example:

```python
from typing import List

def my_function(numbers: List[int]) -> int:
    """
    Calculate the sum of a list of integers.

    Args:
        numbers (List[int]): The list of integers to sum.

    Returns:
        int: The sum of the integers in the list.
    """
    return sum(numbers)
```
[User]: Tell me one tip of JAVA
[Assistant]:  **Tip:** Utilize the `switch` statement for cleaner and more concise conditional logic.

Java 14 introduced the `switch` expression, which a

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)